<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/Evaluation0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pickle
from tqdm import tqdm

In [2]:
entity_df = pd.read_excel('/content/drive/MyDrive/data_freebase/Linking.xlsx')
relation_df = pd.read_excel('/content/drive/MyDrive/data_freebase/dbg.xlsx')
predictions = pd.merge(entity_df, relation_df, on='Question').drop_duplicates()
reverb_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test.xlsx')[['Question', 'triple', 'Reverb_no']]
reverb_df['Relation'] = reverb_df['triple'].apply(lambda item:eval(item)[1])
reverb_df = reverb_df[['Question', 'Relation', 'Reverb_no']]
freebase_df = pd.read_excel('/content/drive/MyDrive/data_freebase/test_useful_records.xlsx')[['Question', 'relation_type']]
freebase_df['Relation'] = freebase_df['relation_type']
freebase_df['Reverb_no'] = freebase_df['relation_type'].apply(lambda item:'')
freebase_df = freebase_df[['Question', 'Relation', 'Reverb_no']]
actuals = pd.concat([reverb_df, freebase_df])

In [3]:
data = pd.merge(actuals, predictions, on='Question', how='inner')
data.head()

,Question,Relation,Reverb_no,NER,Answer,HIT@,Candidates,Freebase,Reverb
0,how do Protocols vary,vary by,267285,protocols,2a6e3082f7bd2a56b001fbb1624e361e5599002d,3,"[(('aabf7ceaade498b555088ad543b63783ca0b4cbd',...","[('fb:medicine.disease.medical_specialties', 0...","{""do n't vary based on"": 0.84525030966336, 'do..."
1,of what is A Shooting Star another brand,is another brand of,8600,a shooting star,e78fa772e4525fa8db441c96351be76ff6a53c2d,1,"[(('e78fa772e4525fa8db441c96351be76ff6a53c2d',...","[('fb:automotive.model.successor', 0.043253843...","{'is another brand of': 1.0000000000000002, 'i..."
2,where was the hometown of Angie Debo,was the hometown of,212931,angie debo,d6d4304250a9caa525fd9630d1cab5ef052d8227,1,"[(('d6d4304250a9caa525fd9630d1cab5ef052d8227',...","[('fb:music.artist.origin', 0.861007), ('fb:pe...","{'was the hometown of': 1.0000000000000002, 'i..."
3,where Curium does not occur,does not occur in,89850,curium,fb:m.025tkv0,1,"[(('fb:m.025tkv0', 'curium', '0.95117000000000...",[('fb:theater.theater_production.performed_at'...,"{'does not occur in': 0.986554414128868, 'does..."
4,who died in Moraga,died in,385331,moraga,23a620d63499f12d9d214118beabc1f586b85c45,1,"[(('23a620d63499f12d9d214118beabc1f586b85c45',...","[('fb:location.location.people_born_here', 0.8...","{'died in': 1.0000000000000002, 'Died in': 1.0..."


In [4]:
reverb2freebace = pd.read_csv('/content/drive/MyDrive/reverb2freebase.csv')
reverb2freebace['freebase_ID_argument1'] = reverb2freebace['freebase_ID_argument1'].apply(lambda string:'fb:m.'+str(string))
reverb2freebace['conf'] = reverb2freebace['conf'].astype(float)
# reverb2freebace.info()

In [5]:
# mapping between MIDs and names in the form of dict['MID']=['str1', 'str2', ...,  'strN']
with open('/content/drive/MyDrive/indexes/names_2M.pkl', 'rb') as f:
    mid2name = pickle.load(f)
# mapping between MIDs and Relations in the form of dict['MID']=[{'fb:common.topic.notable_types', 'fb:people.person.gender', 'fb:people.person.profession'}]
with open('/content/drive/MyDrive/indexes/reachability_2M.pkl', 'rb') as f:
    reachability_2M = pickle.load(f)
print(len(reachability_2M.keys()), sum([len(item) for item in reachability_2M.values()]))

1959805 7188598


In [6]:
def combine(reachability_2M, reverb2freebace):
  for index, row in tqdm(reverb2freebace.iterrows(), total=reverb2freebace.shape[0]):
    if row['freebase_ID_argument1'] in mid2name:
      mid1 = mid = row['freebase_ID_argument1'] 
    else:
      mid1 = mid = row['argument1_uuid']
    mid2 = mid = row['argument2_uuid']
    relation = row['rel']
    reverb_no = row['reverb_no']
    try:
      temp = reachability_2M[mid1]
    except:
      reachability_2M[mid1] = set()
      temp = reachability_2M[mid1]
    temp.add((relation, reverb_no))
    try:
      temp = reachability_2M[mid2]
    except:
      reachability_2M[mid2] = set()
      temp = reachability_2M[mid2]
    temp.add((relation, reverb_no))
  return reachability_2M


In [7]:
reachability_2M = combine(reachability_2M, reverb2freebace)
print(len(reachability_2M.keys()), sum([len(item) for item in reachability_2M.values()]))

100%|██████████| 407267/407267 [00:47<00:00, 8645.23it/s]


2558314 7994311


In [8]:
def create_candidates(ner, freebase, reverb, reverb_no):
  candidates = []
  rvb = [(key, value) for key, value in eval(reverb).items()]
  rels = eval(freebase)+rvb
  print(rels)
  for idx1, ((mid, string, conf), sim1) in enumerate(eval(ner)):
    for idx2, (rel, sim2) in enumerate(rels):
      if (rel in reachability_2M[mid]) or (rel, reverb_no) in reachability_2M[mid]:
        candidates.append((mid, rel, reverb_no, sim1, sim2, conf))
  
  print(len(candidates))

In [9]:
test = data.loc[1, 'Candidates']
create_candidates(data.loc[1, 'Candidates'], data.loc[1, 'Freebase'], data.loc[1, 'Reverb'], data.loc[1, 'Reverb_no'])

[('fb:automotive.model.successor', 0.043253843), ('fb:digicams.camera_storage_type.compatible_cameras', 0.027522288), ('fb:base.cars_refactor.make.model_s', 0.025800252), ('fb:automotive.make.model_s', 0.02205914), ('fb:business.brand.products', 0.012975082), ('fb:aviation.aircraft_manufacturer.aircraft_models_made', 0.011694424), ('fb:food.dietary_restriction.compatible_ingredients', 0.011169725), ('fb:aviation.aircraft_model.variants', 0.01048043), ('fb:metropolitan_transit.transit_vehicle.transit_lines', 0.0095033655), ('fb:location.location.containedby', 1.15016306e-07), ('is another brand of', 1.0000000000000002), ('is a brand of', 0.756433795507315), ('Is A Brand Of', 0.756433795507315), ('is brand of', 0.756433795507315), ('was a brand of', 0.7103702121643718), ('is another of', 0.7092509505437823), ('is also a brand of', 0.6903382167815799), ('is a brand for', 0.6733766492686785), ('are a brand of', 0.6695344523986342), ('is now a brand of', 0.6252908129162552), ('is a common b

KeyError: ignored

In [7]:
data.loc[1, 'Freebase']

"[('fb:automotive.model.successor', 0.043253843), ('fb:digicams.camera_storage_type.compatible_cameras', 0.027522288), ('fb:base.cars_refactor.make.model_s', 0.025800252), ('fb:automotive.make.model_s', 0.02205914), ('fb:business.brand.products', 0.012975082), ('fb:aviation.aircraft_manufacturer.aircraft_models_made', 0.011694424), ('fb:food.dietary_restriction.compatible_ingredients', 0.011169725), ('fb:aviation.aircraft_model.variants', 0.01048043), ('fb:metropolitan_transit.transit_vehicle.transit_lines', 0.0095033655), ('fb:location.location.containedby', 1.15016306e-07)]"

In [8]:
data.loc[1, 'Reverb']

"{'is another brand of': 1.0000000000000002, 'is a brand of': 0.756433795507315, 'Is A Brand Of': 0.756433795507315, 'is brand of': 0.756433795507315, 'was a brand of': 0.7103702121643718, 'is another of': 0.7092509505437823, 'is also a brand of': 0.6903382167815799, 'is a brand for': 0.6733766492686785, 'are a brand of': 0.6695344523986342, 'is now a brand of': 0.6252908129162552, 'is a common brand of': 0.6154475666799368, 'is a brand name of': 0.6139686011945599, 'is a popular brand of': 0.611194214286004, 'is a major brand of': 0.608077002536472, 'is another part of': 0.6062298786823834, 'is another member of': 0.6057647604678563, 'is the brand name of': 0.586868342879729, 'was a brand name of': 0.5782827351471661, 'is an American brand of': 0.5693266654055181, 'was another member of': 0.5653552201448889, 'is another area of': 0.5636348252922414, 'is another name of': 0.5626276387457763, 'is another form of': 0.557189718644548, 'is a brand name for': 0.5528358483310428, 'is brand n

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
